In [1]:
import sys
import os
import numpy as np
import importlib

# hacky way, figure out proper way later
import pickle as pkl
import re

In [2]:
sys.path.append(os.path.abspath("../"))

In [3]:
import utils.hooklessrun

importlib.reload(utils.hooklessrun)

<module 'utils.hooklessrun' from '/Users/dkundu/Documents/phd/ann-rnn-modified/utils/hooklessrun.py'>

In [4]:
train_run_id = "fixed_time_stim_no_reward"  # new training run id
run_dir = "/Users/dkundu/Documents/phd/ann-rnn-modified/runs/"
train_run_dir = os.path.join(run_dir, train_run_id)
analyze_run_dir = os.path.join(train_run_dir, "analyze")

checkpoint_paths = [
    os.path.join(train_run_dir, file_path)
    for file_path in os.listdir(train_run_dir)
    if file_path.endswith(".pt")
]

checkpoints = np.sort(checkpoint_paths)

In [5]:
# silly but numeric sort for checkpoint files

numbered_files = [f for f in checkpoint_paths if re.search(r"\d+.pt$", f)]
sorted_files = sorted(
    numbered_files, key=lambda x: int(re.search(r"(\d+).pt$", x).group(1))
)

In [6]:
run_dir

'/Users/dkundu/Documents/phd/ann-rnn-modified/runs/'

In [7]:
# for idx in range():

setup_results_99000 = utils.hooklessrun.setup_analyze(
    run_dir=run_dir, train_run_id=train_run_id, checkpoint_number="99000"
)

setup_results_3000 = utils.hooklessrun.setup_analyze(
    run_dir=run_dir, train_run_id=train_run_id, checkpoint_number="3000"
)

setup_results_0 = utils.hooklessrun.setup_analyze(
    run_dir=run_dir, train_run_id=train_run_id, checkpoint_number="0000"
)

Seed: 1
Loading checkpoint at /Users/dkundu/Documents/phd/ann-rnn-modified/runs/fixed_time_stim_no_reward/checkpoint_grad_steps=99000.pt
rnn, input_mask=none, readout_mask=none, recurrent_mask=none, hidden_size=10, num_layers=1, param_init=default


/Users/dkundu/Documents/phd/ann-rnn-modified/utils/hooklessrun.py:246: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  save_dict = torch.load(checkpoint_path)


Logger created successfully
Seed: 1
Seed: 1
Loading checkpoint at /Users/dkundu/Documents/phd/ann-rnn-modified/runs/fixed_time_stim_no_reward/checkpoint_grad_steps=3000.pt
Loading checkpoint at /Users/dkundu/Documents/phd/ann-rnn-modified/runs/fixed_time_stim_no_reward/checkpoint_grad_steps=3000.pt
rnn, input_mask=none, readout_mask=none, recurrent_mask=none, hidden_size=10, num_layers=1, param_init=default
Logger created successfully
Logger created successfully
Seed: 1
Seed: 1
Seed: 1
Loading checkpoint at /Users/dkundu/Documents/phd/ann-rnn-modified/runs/fixed_time_stim_no_reward/checkpoint_grad_steps=0000.pt
Loading checkpoint at /Users/dkundu/Documents/phd/ann-rnn-modified/runs/fixed_time_stim_no_reward/checkpoint_grad_steps=0000.pt
Loading checkpoint at /Users/dkundu/Documents/phd/ann-rnn-modified/runs/fixed_time_stim_no_reward/checkpoint_grad_steps=0000.pt
rnn, input_mask=none, readout_mask=none, recurrent_mask=none, hidden_size=10, num_layers=1, param_init=default


/Users/dkundu/Documents/phd/ann-rnn-modified/utils/hooklessrun.py:246: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  save_dict = torch.load(checkpoint_path)


In [8]:
# # # fix this error, step through and debug
# # # should be easy
# run_envs_output = utils.hooklessrun.run_envs(
#     model=setup_results_3000["model"], envs=setup_results_3000["envs"]
# )

{'connectivity_kwargs': {'input_mask': 'none',
  'readout_mask': 'none',
  'recurrent_mask': 'none'},
 'core_kwargs': {'hidden_size': 10, 'num_layers': 1},
 'input_size': 3,
 'output_size': 2,
 'param_init': 'default'}

In [9]:
for name, param in setup_results_3000["model"].named_parameters():
    print(name, param.shape)

core.weight_ih_l0 torch.Size([10, 3])
core.weight_hh_l0 torch.Size([10, 10])
core.bias_ih_l0 torch.Size([10])
core.bias_hh_l0 torch.Size([10])
readout.weight torch.Size([2, 10])
readout.bias torch.Size([2])


In [25]:
for param in setup_results_0["model"].core.parameters():
    param.requires_grad = False


for name, param in setup_results_0["model"].named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

core.weight_ih_l0: requires_grad=False
core.weight_hh_l0: requires_grad=False
core.bias_ih_l0: requires_grad=False
core.bias_hh_l0: requires_grad=False
readout.weight: requires_grad=True
readout.bias: requires_grad=True


In [24]:
import torch
from scipy.stats import pearsonr

In [47]:
pearsonr(
    setup_results_0["model"].core.weight_hh_l0.detach().numpy().flatten(),
    setup_results_99000["model"].core.weight_hh_l0.detach().numpy().flatten(),
)

PearsonRResult(statistic=0.8914661451892587, pvalue=1.8189704832639683e-35)